# Experiments

In [39]:
import pandas as pd
import numpy as np
from modules.functions import run_linear_regression
from modules.functions import run_polynomial_regression
from modules.functions import run_random_forest

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# import csv file with the proper dtypes
dtype_dict = {
 'branded_code_share': 'string',
 'mkt_carrier': 'string',
 'mkt_carrier_fl_num': 'string',
 'op_unique_carrier': 'string',
 'tail_num': 'string',
 'op_carrier_fl_num': 'string',
 'origin_airport_id': 'string',
 'origin': 'string',
 'origin_city_name': 'string',
 'dest_airport_id': 'string',
 'dest': 'string',
 'dest_city_name': 'string',
 'crs_dep_time': 'int64',
 'dep_time': 'int64',
 'dep_delay': 'int64',
 'taxi_out': 'int64',
 'wheels_off': 'int64',
 'wheels_on': 'int64',
 'taxi_in': 'int64',
 'crs_arr_time': 'int64',
 'arr_time': 'int64',
 'arr_delay': 'int64',
 'crs_elapsed_time': 'int64',
 'actual_elapsed_time': 'int64',
 'air_time': 'int64',
 'distance': 'int64'}
df = pd.read_csv("../LHL_Midterm_Project_Predicting_Flight_Delays/data/ransmpl_clean.csv", parse_dates=[0], dtype=dtype_dict)
df_no_outliers = pd.read_csv("../LHL_Midterm_Project_Predicting_Flight_Delays/data/ransmpl_clean_no_outliers.csv", parse_dates=[0], dtype=dtype_dict)

In [4]:
df.head()

,fl_date,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,crs_elapsed_time,actual_elapsed_time,air_time,distance
0,2018-03-07,AA,AA,465,AA,N200UU,465,14107,PHX,"Phoenix, AZ",14679,SAN,"San Diego, CA",835,833,-2,13,846,838,2,851,840,-11,76,67,52,304
1,2018-03-07,AA,AA,591,AA,N833AW,591,11057,CLT,"Charlotte, NC",11278,DCA,"Washington, DC",1431,1537,66,16,1553,1648,3,1559,1651,52,88,74,55,331
2,2018-03-07,AA,AA,600,AA,N151UW,600,11697,FLL,"Fort Lauderdale, FL",11057,CLT,"Charlotte, NC",603,557,-6,18,615,746,19,809,805,-4,126,128,91,632
3,2018-03-07,AA,AA,1805,AA,N924US,1805,11057,CLT,"Charlotte, NC",10721,BOS,"Boston, MA",1135,1129,-6,11,1140,1312,12,1352,1324,-28,137,115,92,728
4,2018-03-07,AA,AA,2615,AA,N945NN,2615,11057,CLT,"Charlotte, NC",15370,TUL,"Tulsa, OK",1820,1812,-8,11,1823,1936,6,2002,1942,-20,162,150,133,842


In [5]:
df2 = df[['fl_date', 'branded_code_share', 'mkt_carrier', 'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num', 'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name', 'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'crs_arr_time', 'crs_elapsed_time', 'distance']].copy()
df2.head()

,fl_date,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,distance
0,2018-03-07,AA,AA,465,AA,N200UU,465,14107,PHX,"Phoenix, AZ",14679,SAN,"San Diego, CA",835,851,76,304
1,2018-03-07,AA,AA,591,AA,N833AW,591,11057,CLT,"Charlotte, NC",11278,DCA,"Washington, DC",1431,1559,88,331
2,2018-03-07,AA,AA,600,AA,N151UW,600,11697,FLL,"Fort Lauderdale, FL",11057,CLT,"Charlotte, NC",603,809,126,632
3,2018-03-07,AA,AA,1805,AA,N924US,1805,11057,CLT,"Charlotte, NC",10721,BOS,"Boston, MA",1135,1352,137,728
4,2018-03-07,AA,AA,2615,AA,N945NN,2615,11057,CLT,"Charlotte, NC",15370,TUL,"Tulsa, OK",1820,2002,162,842


# Feature Engineering

In [6]:
import datetime
import calendar

In [7]:
dir(calendar)

['Calendar',
 'EPOCH',
 'FRIDAY',
 'February',
 'HTMLCalendar',
 'IllegalMonthError',
 'IllegalWeekdayError',
 'January',
 'LocaleHTMLCalendar',
 'LocaleTextCalendar',
 'MONDAY',
 'SATURDAY',
 'SUNDAY',
 'THURSDAY',
 'TUESDAY',
 'TextCalendar',
 'WEDNESDAY',
 '_EPOCH_ORD',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_colwidth',
 '_locale',
 '_localized_day',
 '_localized_month',
 '_monthlen',
 '_nextmonth',
 '_prevmonth',
 '_spacing',
 'c',
 'calendar',
 'datetime',
 'day_abbr',
 'day_name',
 'different_locale',
 'error',
 'firstweekday',
 'format',
 'formatstring',
 'isleap',
 'leapdays',
 'main',
 'mdays',
 'month',
 'month_abbr',
 'month_name',
 'monthcalendar',
 'monthrange',
 'prcal',
 'prmonth',
 'prweek',
 'repeat',
 'setfirstweekday',
 'sys',
 'timegm',
 'week',
 'weekday',
 'weekheader']

In [8]:
help(calendar.firstweekday)

Help on method getfirstweekday in module calendar:

getfirstweekday() method of calendar.TextCalendar instance



In [9]:
df2["year"] = df2["fl_date"].dt.year.astype("object")
df2["month"] = df2["fl_date"].dt.month.astype("object")
df2["day"] = df2["fl_date"].dt.day.astype("object")
df2["day_of_week"] = df2["fl_date"].dt.weekday.astype("object")

In [10]:
df2["fl_date"].dt.weekday

0         2
1         2
2         2
3         2
4         2
         ..
390053    2
390054    2
390055    2
390056    2
390057    2
Name: fl_date, Length: 390058, dtype: int64

In [11]:
df2.head()

,fl_date,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,year,month,day,day_of_week
0,2018-03-07,AA,AA,465,AA,N200UU,465,14107,PHX,"Phoenix, AZ",14679,SAN,"San Diego, CA",835,851,76,304,2018,3,7,2
1,2018-03-07,AA,AA,591,AA,N833AW,591,11057,CLT,"Charlotte, NC",11278,DCA,"Washington, DC",1431,1559,88,331,2018,3,7,2
2,2018-03-07,AA,AA,600,AA,N151UW,600,11697,FLL,"Fort Lauderdale, FL",11057,CLT,"Charlotte, NC",603,809,126,632,2018,3,7,2
3,2018-03-07,AA,AA,1805,AA,N924US,1805,11057,CLT,"Charlotte, NC",10721,BOS,"Boston, MA",1135,1352,137,728,2018,3,7,2
4,2018-03-07,AA,AA,2615,AA,N945NN,2615,11057,CLT,"Charlotte, NC",15370,TUL,"Tulsa, OK",1820,2002,162,842,2018,3,7,2


In [37]:
pd.to_timedelta(df2["crs_dep_time"], unit="h").dt.components

,days,hours,minutes,seconds,milliseconds,microseconds,nanoseconds
0,34,19,0,0,0,0,0
1,59,15,0,0,0,0,0
2,25,3,0,0,0,0,0
3,47,7,0,0,0,0,0
4,75,20,0,0,0,0,0
...,...,...,...,...,...,...,...
390053,60,10,0,0,0,0,0
390054,51,21,0,0,0,0,0
390055,66,23,0,0,0,0,0
390056,79,4,0,0,0,0,0


In [12]:
df_num = df2[["month", "day_of_week"]]
df_num = pd.get_dummies(df_num)
df_num

,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390053,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
390054,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
390055,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
390056,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


# Select Features

In [19]:
y = df["arr_delay"]
y2 = df_no_outliers["arr_delay"]

X1 = df[["air_time", "distance"]]
X2 = df_num
X3 = df_no_outliers[["air_time", "distance"]] # current best

# Test Models

In [25]:
# Linear Regression
y_pred, r2_train, r2_test = run_linear_regression(X3, y2)
print(r2_train, r2_test)

0.029350330153553594 0.030463382969991293


In [21]:
# Polynomial Regression
for degree in range(2,10):
    y_pred, r2_train, r2_test = run_polynomial_regression(X3, y2, degree)

---Results---
degree = 2
Train score = 0.03833909841202254
Test score = 0.035460115211514376
---Results---
degree = 3
Train score = 0.04047064977269843
Test score = 0.040914132301534956
---Results---
degree = 4
Train score = 0.04233778531937615
Test score = 0.042495701123806184
---Results---
degree = 5
Train score = 0.042826042700197076
Test score = 0.03999902076952666
---Results---
degree = 6
Train score = 0.04201955850851791
Test score = 0.04176713311520697
---Results---
degree = 7
Train score = 0.033933805079104595
Test score = -5.603117502750443e-05
---Results---
degree = 8
Train score = 0.022134094578955543
Test score = -0.013338066127134951
---Results---
degree = 9
Train score = 0.01613739449438567
Test score = 0.002090088405743651


In [22]:
# Random Forest
y_pred, r2_train, r2_test = run_random_forest(X3, y2)
print(r2_train, r2_test)

train score = 0.08063265973094125
test score = 0.03698613774569015
0.08063265973094125 0.03698613774569015
